## Solve using Benders

In [1]:
from gurobipy import *

In [2]:
import qminos
from qminos import qwarmLP

In [3]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

In [4]:
from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders

In [5]:
from cobra.io import load_json_model

ijomc  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')
ijofit = load_json_model('/home/laurence/ME/data/dynamicME/best_ijomc.json')

In [6]:
from dynamicme.optimize import Optimizer

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f18f3d24128>)


In [7]:
opt = Optimizer(ijomc)

In [8]:
gap = opt.add_duality_gap_constraint()

In [9]:
crowding = ijomc.metabolites.crowding
crowdingfit = ijofit.metabolites.crowding
a12_dict = {(gap.metabolites.get_by_id(crowding.id), gap.reactions.get_by_id(rxn.id)):(
    rxn.metabolites[crowding], 
    ijofit.reactions.get_by_id(rxn.id).metabolites[crowdingfit]) for rxn in crowding.reactions}

In [10]:
opt.make_disjunctive_primal_dual(gap, a12_dict)

<Model duality_gap at 0x7f18f56c7fd0>

In [11]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
for rxn in gap.reactions.query('binary_'):
    rxn.objective_coefficient = 1.

In [12]:
ijofit.optimize()
MU_FIT = ijofit.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

In [13]:
MU_FIT

0.7301837182736599

In [14]:
F_TOL = 0.05

gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = MU_FIT*(1+F_TOL)
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.lower_bound = MU_FIT*(1-F_TOL)

In [15]:
gap.optimize(solver='gurobi', objective_sense='minimize')

<Solution 2.00 at 0x7f18f04d9a10>

In [16]:
sum([r.x for r in gap.reactions.query('binary_')])

2.0

In [17]:
bvars = [r for r in gap.reactions if r.variable_kind == 'integer']
len(bvars)

2018

In [18]:
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7549193132844437

In [19]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        print rxn, rxn.x

binary_crowding_GK1_abs 1.0
binary_crowding_ATPS4rpp_abs 1.0


## Solve as MILP

In [20]:
from cobra.solvers import gurobi_solver
from gurobipy import *

In [21]:
%%time
milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE

CPU times: user 332 ms, sys: 15.7 ms, total: 348 ms
Wall time: 319 ms


In [22]:
milp.Params.IntFeasTol = 1e-9
milp.Params.FeasibilityTol = 1e-8
milp.Params.OptimalityTol = 1e-8
milp.Params.Presolve = 0
milp.Params.NumericFocus = 1

In [23]:
milp.Params.OutputFlag=1
%time milp.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 22283 rows, 18524 columns and 79543 nonzeros
Variable types: 16506 continuous, 2018 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-11, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 1e+03]
  RHS range        [5e-03, 1e+04]
Variable types: 16387 continuous, 2137 integer (2018 binary)

Root relaxation: objective 6.221914e-02, 21372 iterations, 3.53 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.06222    0    1          -    0.06222      -     -    3s
     0     0    0.08374    0    3          -    0.08374      -     -    3s
     0     2    0.78209    0    3          -    0.78209      -     -    4s
H   12    16                       1.0000000    0.83447  16.6%  62.9    4s

Cutting planes:
  MIR: 1

Explored 15 nodes (2

In [24]:
milp.Status == GRB.OPTIMAL

True

In [25]:
milp.ObjVal

1.0

In [26]:
sol = gurobi_solver.format_solution(milp, gap)

In [27]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        xmilp = sol.x_dict[rxn.id]
        print rxn, rxn.x, xmilp

binary_crowding_GK1_abs 1.0 -0.0
binary_crowding_ATPS4rpp_abs 1.0 1.0


## Solve using Benders with quad/dqq subproblem

In [ ]:
from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders

decomposer = Decomposer(milp)
master,sub = decomposer.benders_decomp()
master._precision_sub = 'quad'
master._verbosity = 1

In [33]:
%%time
master.optimize(cb_benders)

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05
Optimize a model with 1 rows, 2019 columns and 2019 nonzeros
Variable types: 1 continuous, 2018 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [0e+00, 0e+00]
Getting MINOS parameters...
Done in 303.436 seconds with status 0
zmaster=0. zsub=3.70303. gap=-3.70303
Getting MINOS parameters...
Done in 1.32742 seconds with status 0
zmaster=0. zsub=3.70303. gap=-3.70303
Getting MINOS parameters...
Done in 1.3002 seconds with status 0
zmaster=0. zsub=3.70303. gap=-3.70303
Variable types: 1 continuous, 2018 integer (2018 binary)
Getting MINOS parameters...
Done in 1.30425 seconds with status 0
zmaster=0. zsub=3.703

In [54]:
import numpy as np
yopt = np.array([y.X for y in decomposer._ys])
sum(yopt)

1.0

In [55]:
y_dict = {y.VarName:y.X for y in decomposer._ys}

In [35]:
from qminos.quadLP import QMINOS
### SOlve primal sub
qsolver = QMINOS()
sense_dict = {GRB.EQUAL:'E', GRB.LESS_EQUAL:'L', GRB.GREATER_EQUAL:'G'}
csenses = [sense_dict[c] for c in decomposer._csenses]
xp,statp,hsp = qsolver.solvelp(decomposer._A, decomposer._d - decomposer._B*yopt, decomposer._cx, decomposer._xl, decomposer._xu, csenses, precision='dqq')

Getting MINOS parameters...
Done in 30.8749 seconds with status 0


In [36]:
x_dict = {x.VarName:xp[j] for j,x in enumerate(decomposer._x0)}

In [39]:
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7549193132844437

In [37]:
milp.getVars()[7]

<gurobi.Var 7 (value 0.749027199175)>

In [38]:
x_dict['7']

0.7490271991752928

In [41]:
print(gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.lower_bound, gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound)

(0.6936745323599769, 0.7666929041873429)


In [52]:
rids = [r.id for r in gap.reactions]
yid_dict = {y.VarName:rids[int(y.VarName)] for y in decomposer._y0}
y_inds = {rids[int(y.VarName)]:y.VarName for y in decomposer._y0}
y_inds

{u'binary_crowding_GLCURtex_abs': '14867',
 u'binary_crowding_SPODM': '16757',
 u'binary_crowding_THRS': '17657',
 u'binary_crowding_DINSt2pp': '17840',
 u'binary_crowding_THRD': '15107',
 u'binary_crowding_RPI_abs': '14888',
 u'binary_crowding_AMALT3': '17150',
 u'binary_crowding_DHPPD': '14216',
 u'binary_crowding_METSOX1abcpp': '15527',
 u'binary_crowding_DAPAL': '17735',
 u'binary_crowding_ACACCT': '16277',
 u'binary_crowding_MINCYCtpp': '17132',
 u'binary_crowding_G3PCabcpp': '16331',
 u'binary_crowding_AGM4PCPpp': '14462',
 u'binary_crowding_TTRCYCtex_abs': '13358',
 u'binary_crowding_PAPA140': '12668',
 u'binary_crowding_PAPA141': '14492',
 u'binary_crowding_TRSARr_abs': '13508',
 u'binary_crowding_FE3HOXexs': '17966',
 u'binary_crowding_GGGABAH': '17627',
 u'binary_crowding_GLYBt2pp': '18002',
 u'binary_crowding_TRPt2rpp_abs': '14456',
 u'binary_crowding_FUSAtpp': '17936',
 u'binary_crowding_GLCDpp': '14927',
 u'binary_crowding_PLIPA1A120pp': '17306',
 u'binary_crowding_PUNP2_a

In [59]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        xmilp = sol.x_dict[rxn.id]
        xbend = y_dict[y_inds[rxn.id]]
        print rxn, rxn.x, xmilp, xbend

binary_crowding_GK1_abs 1.0 -0.0 -0.0
binary_crowding_ATPS4rpp_abs 1.0 1.0 1.0
